In [ ]:
import csv
import itertools as it
import pandas as pd
import numpy as np
import sklearn.decomposition
from tqdm import tqdm
import re
import string
import random
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import numpy.lib.stride_tricks as tri
import os
from datetime import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as torch_data
from torch.autograd import Variable
import scipy.optimize
from functools import partial
from numpy import linalg as LA

: 

In [ ]:
def mysinc(x):
    return torch.sinc(x / torch.pi)

In [ ]:
def calc_qsp_coeff_tensor(phases):
  # print("Entered: calc_qsp_coeff_tensor")
  # print(phases, degree)
  # print("****************")

  degree = int((len(phases)-3)/2)
  # print("degree is", degree)

  theta_list = torch.tensor(phases[:degree+1])
  phi_list= torch.tensor(phases[degree+1:2*degree+2])
  lambda0 = torch.tensor([phases[-1]])

  # print("theta_list:", theta_list)
  # print("phi_list:", phi_list)
  # print("lambda0:", lambda0)

  pcoeff = torch.zeros(2 * degree + 1, dtype = torch.double)
  qcoeff = torch.zeros(2 * degree + 1, dtype = torch.double)

  if degree == 0:
    pcoeff[0] = torch.exp(torch.tensor(1j*(phi_list[0] + lambda0[0])))*torch.cos(torch.tensor(theta_list[0]))
    qcoeff[0] = torch.exp(torch.tensor(1j*lambda0[0]))*torch.sin(torch.tensor(theta_list[0]))
  else:
    theta_reduce = torch.tensor(theta_list[:-1])
    phi_reduce = torch.tensor(phi_list[:-1])

    phases_reduce = torch.cat((theta_reduce, phi_reduce, lambda0))

    pcoeff_reduce, qcoeff_reduce = calc_qsp_coeff_tensor(phases_reduce)
    # now compute the new coefficients from the old one
    for idx in range(-degree, degree + 1):
        # print("idx = ", idx)
        if idx >= degree - 1:
            # print(idx + degree - 2)
            if idx - 1 >= 0:
                # note the index in the RHS has to be idx+degree-2, instead of idx+degree-1 because there is also one negative index coefficient less.
                pcoeff[idx + degree] = torch.exp(torch.tensor(1j*phi_list[-1]))*torch.cos(torch.tensor(theta_list[-1])) * pcoeff_reduce[idx + degree - 2]
                qcoeff[idx + degree] = torch.sin(torch.tensor(theta_list[-1])) * pcoeff_reduce[idx + degree - 2]

        elif idx <= -(degree - 1):
            if idx + 1 <= 0:
                pcoeff[idx + degree] = torch.exp(torch.tensor(1j*phi_list[-1]))*torch.sin(torch.tensor(theta_list[-1])) * qcoeff_reduce[idx + degree]
                qcoeff[idx + degree] = -torch.cos(torch.tensor(theta_list[-1])) * qcoeff_reduce[idx + degree]

        elif abs(idx) <= degree - 2:
            pcoeff[idx + degree] = torch.exp(torch.tensor(1j*phi_list[-1]))*(torch.cos(torch.tensor(theta_list[-1])) * pcoeff_reduce[idx + degree - 2] \
                                + torch.sin(torch.tensor(theta_list[-1])) * qcoeff_reduce[idx + degree])
            qcoeff[idx + degree] = torch.sin(torch.tensor(theta_list[-1])) * pcoeff_reduce[idx + degree - 2] \
                                - torch.cos(torch.tensor(theta_list[-1])) * qcoeff_reduce[idx + degree]

        else:
            print("something wrong with indexing. check.")

  return pcoeff, qcoeff

In [ ]:
def signal_poly_coeff(phases, K, sigma):
    # print("Entered: signal_poly_coeff")
    # print(phases, K, sigma, degree)
    # print("****************")

    # make torch tensors to store the coefficients of the GQSP
    degree = int((len(phases)-3)/2)
    # print("degree in signal_poly_coeff", degree)

    pcoeff = torch.zeros(2 * degree + 1, dtype = torch.complex64)
    qcoeff = torch.zeros(2 * degree + 1, dtype = torch.complex64)
    ccoeff = torch.zeros(2 * degree + 1, dtype = torch.complex64)

    # compute the GQSP coefficients using phases

    pcoeff, qcoeff = calc_qsp_coeff_tensor(phases)

    for s in range(-degree, degree + 1):
        cs = 0.0
        for t in range(-degree, degree + 1):
            for n in range(-degree, degree + 1):
                 for nq in range(-degree, degree + 1):
                    if abs(n + 2 * s) <= degree and abs(nq + 2 * t) <= degree:
                        cs += (pcoeff[n + degree] * torch.conj(pcoeff[nq + degree]) + qcoeff[n + degree] * torch.conj(qcoeff[nq + degree])) \
                            * (torch.conj(pcoeff[n + 2 * s + degree]) * pcoeff[nq + 2 * t + degree] + torch.conj(qcoeff[n + 2 * s + degree]) * qcoeff[nq + 2 * t + degree]) \
                            * torch.exp(torch.tensor(-K ** 2 * sigma ** 2 * (t - s) ** 2))

        ccoeff[s + degree] = cs
    # print("ccoeff", ccoeff)
    return ccoeff

In [ ]:
# signal_poly_coeff(phases, K, sigma, degree)

In [ ]:
def loss_fn_exact(phases, beta_th_p, beta_th_n, K, sigma, flag_callback = False):

    degree = int((len(phases)-3)/2)
    # make torch tensors to store the coefficients of the GQSP

    #initialize the coefficient vectors
    #for even degree, p & q have 2d+1 terms, for odd degree, 2d terms
    pcoeff = torch.zeros(2 * degree + 1, dtype = torch.complex64)
    qcoeff = torch.zeros(2 * degree + 1, dtype = torch.complex64)

    # compute the GQSP coefficients using phases

    pcoeff, qcoeff = calc_qsp_coeff_tensor(phases)

    # compute p_err from the coefficients analytically
    p_err = 0.0
    for s in range(-2 * degree, 2 * degree + 1):

      # this is range of integration for [-pi / 2k, pi / 2k]
      Hs = mysinc(torch.tensor(torch.pi * s)) \
      + 1j*(torch.exp(torch.tensor(1j*2*K * beta_th_p*s)) - torch.exp(torch.tensor(1j*2*K * beta_th_n*s)))

      cs = 0.0
      for n in range(-degree, degree + 1):
          if abs(n + s) <= degree:
              for mp in range(-degree, degree + 1):
                  for nq in range(-degree, degree + 1):
                      cs += (pcoeff[n + degree] * torch.conj(pcoeff[nq + degree]) + qcoeff[n + degree] * torch.conj(qcoeff[nq + degree])) \
                      * (torch.conj(pcoeff[n + s + degree]) * pcoeff[mp + degree] + torch.conj(qcoeff[n + s + degree]) * qcoeff[mp + degree]) \
                      * torch.exp(torch.tensor(-0.25 * K ** 2 * sigma ** 2 * (mp - nq - s) ** 2))

      p_err += cs * Hs
    p_err = p_err + K * (beta_th_p - beta_th_n) / torch.pi
    if flag_callback:
        print(" => " + str(datetime.now()) + ": loss = ", p_err.item(), "  Phases = ", phases, " Ccoeff = ", signal_poly_coeff(phases, K, sigma))
    # print("p_err", p_err)
    return p_err.real

In [ ]:
np.random.seed()

thresh = 1e-5
tol = 1e-8
# torch.manual_seed(0)

K = 1 / 2048  #kappa
sigma = 1
degree_list = [7]
num_trials = 1
eta = 0.25

beta_th_p = eta * np.pi / K #threshold for positive beta values
beta_th_n = -0.5*eta * np.pi / K #threshold for negative beta values

d_best_results = []
d_best_phases = []
d_best_losses = []

opt_method = 'Nelder-Mead'
options = {'fatol': thresh, 'xatol': thresh}

for degree in degree_list:
    print("Degree", degree)
    best_results = []
    best_phases = []
    best_losses = []
    for trial in range(num_trials):
        print("Trial", trial + 1)
        # Try the protocol with GQSP
        # generates a random list of 2d+1 angles *(b/w 0 to 2pi)

        #initialize the list of theta, phi and lambda0 angle vector lenght: (d+1, d+1, 1)
        thetas = torch.rand(degree + 1) * 2 * torch.pi
        phis = torch.rand(degree + 1) * 2 * torch.pi
        lambda0 = torch.rand(1) * 2 * torch.pi
        # print("initial thetas:", thetas)
        # print("initial phis:", phis)
        # print("initial lambda0:", lambda0)
        angles0 = torch.cat((thetas, phis, lambda0))
        print("angles:", angles0)

        # This is to use the original loss function directly
        new_callback = partial(loss_fn_exact, beta_th_p = beta_th_p, beta_th_n = beta_th_n,  K = K, sigma = sigma, flag_callback = True)
        res = scipy.optimize.minimize(loss_fn_exact, angles0, (beta_th_p, beta_th_n, K, sigma),
                                      method = opt_method, tol = thresh, callback = new_callback, options = options)
        best_results.append(res)
        best_phases.append(res.x)
        best_losses.append(res.fun)
    best_losses = np.array(best_losses)
    d_best_results.append(best_results)
    d_best_phases.append(best_phases)
    d_best_losses.append(best_losses)

for i, phases in enumerate(d_best_phases):
    print("Degree", degree_list[i])
    best_index = np.argmin(d_best_losses[i])
    best_loss = d_best_losses[i][best_index]
    best_phases = phases[best_index]
    print("Best loss for degree " + str(degree_list[i]) + ":", best_loss)
    print("Best phases for degree " + str(degree_list[i]) + ":", best_phases)
    theta_list = phases[0][:degree_list[i]+1]
    phi_list= phases[0][degree_list[i]+1:2*degree_list[i]+2]
    lambda0 = [phases[0][-1]]
    print("theta_list:", theta_list)
    print("phi_list:", phi_list)
    print("lambda0:", lambda0)
